In [1]:
!git clone https://github.com/rosinality/stylegan2-pytorch.git

Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 395, done.
remote: Total 395 (delta 0), reused 0 (delta 0), pack-reused 395 (from 1)
Receiving objects: 100% (395/395), 122.51 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (205/205), done.


In [2]:
cd stylegan2-pytorch

/content/stylegan2-pytorch


In [3]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00


In [4]:
!pip install Ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 22.9 MB/s eta 0:00:00


In [5]:
# Репозиторий StyleGAN-2 использует более старую версию библиотеки skimage, поэтому нам необходимо сделать небольшую корректировку файла с lpips лоссом.
file_path = "/content/stylegan2-pytorch/lpips/__init__.py"

# Откроем файл и прочитаем его содержимое
with open(file_path, 'r') as file:
    lines = file.readlines()

# Заменим нужную строку
with open(file_path, 'w') as file:
    for line in lines:
        # Если в строке есть "compare_ssim", заменим на "structural_similarity"
        if "from skimage.measure import compare_ssim" in line:
            file.write("from skimage.metrics import structural_similarity\n")
        else:
            file.write(line)

print("Замена выполнена успешно!")

import lpips  # Библиотека для вычисления perceptual loss


Замена выполнена успешно!


In [6]:
# Загрузка модели StyleGAN-2
!gdown 1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT

Downloading...
From (original): https://drive.google.com/uc?id=1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT
From (redirected): https://drive.google.com/uc?id=1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT&confirm=t&uuid=a39750ef-36f5-4e95-8a65-254d16353624
To: /content/stylegan2-pytorch/stylegan2-ffhq-config-f.pt
100% 381M/381M [00:11<00:00, 32.1MB/s]


In [7]:
cd /content

/content


In [8]:
# Скачаем стартовое изображение
!wget -P /content/ -O image.jpg https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfN_LL8waaCzWlbneQbQQyL2HSPT74j5_l69bM5TvB1lcJI-tjKsP2PI7uSMN3ooXQMko&usqp=CAU
# %matplotlib inline

--2025-02-03 18:55:31--  https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfN_LL8waaCzWlbneQbQQyL2HSPT74j5_l69bM5TvB1lcJI-tjKsP2PI7uSMN3ooXQMko
Resolving encrypted-tbn0.gstatic.com (encrypted-tbn0.gstatic.com)... 64.233.170.139, 64.233.170.113, 64.233.170.101, ...
Connecting to encrypted-tbn0.gstatic.com (encrypted-tbn0.gstatic.com)|64.233.170.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6406 (6.3K) [image/jpeg]
Saving to: ‘image.jpg’

                    100%[===================>]   6.26K  --.-KB/s    in 0s      

2025-02-03 18:55:31 (72.4 MB/s) - ‘image.jpg’ saved [6406/6406]



In [9]:
# Загружаем модель для определения ключевых точек лица - понадобится нам для выравнивания
import os
experiment_type = "ffhq_encode"
if experiment_type == "ffhq_encode" and 'shape_predictor_68_face_landmarks.dat' not in os.listdir():
    !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2  # Скачиваем архив с моделью
    !bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2  # Распаковываем архив

--2025-02-03 18:55:31--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-02-03 18:55:32--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  11.5MB/s    in 6.5s    

2025-02-03 18:55:39 (9.33 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [10]:
# Загружаем сеть-энкодер для получения латентов StyleGAN без выполнения оптимизации для каждого изображения
! git clone https://github.com/omertov/encoder4editing.git

Cloning into 'encoder4editing'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 172 (delta 49), reused 42 (delta 42), pack-reused 94 (from 1)
Receiving objects: 100% (172/172), 33.43 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [11]:
cd /content/encoder4editing

/content/encoder4editing


In [12]:
# Скачаем модель энкодера
!gdown 1cUv_reLE6k3604or78EranS7XzuVMWeO

Downloading...
From (original): https://drive.google.com/uc?id=1cUv_reLE6k3604or78EranS7XzuVMWeO
From (redirected): https://drive.google.com/uc?id=1cUv_reLE6k3604or78EranS7XzuVMWeO&confirm=t&uuid=ce4e15ee-f197-4295-872a-c05d76f147db
To: /content/encoder4editing/e4e_ffhq_encode.pt
100% 1.20G/1.20G [00:24<00:00, 49.1MB/s]


In [13]:
# ЗАПУСКАЕМ STREAMLIT

In [42]:
%%writefile /content/face_editor.py
import os
os.chdir('/content/stylegan2-pytorch')
import sys
sys.path.append('/content/stylegan2-pytorch')
sys.path.append('/content/encoder4editing')
import torch
import torch.optim as optim
from model import Generator
from torchvision.utils import save_image
from PIL import Image
import numpy as np
from utils.common import tensor2im
import matplotlib.pyplot as plt

import torch.optim as optim
import math  # Модуль для математических операций
from torch.nn import functional as F
from torchvision import transforms  # Модуль для преобразования изображений
from PIL import Image  # Библиотека для работы с изображениями
from tqdm import tqdm

os.chdir('/content/encoder4editing')
from encoder4editing.models.psp import pSp  # Импортируем модель pSp
from argparse import Namespace

torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Параметры генератора
size = 1024  # Размер изображения
latent_dim = 512  # Размер латентного пространства
n_mlp = 8  # Количество слоев MLP
channel_multiplier = 2
ckpt = '/content/stylegan2-pytorch/stylegan2-ffhq-config-f.pt'  # Путь к весам StyleGAN2

# Инициализация генератора
generator = Generator(size, latent_dim, n_mlp, channel_multiplier=channel_multiplier).to(device)
generator.eval()  # Режим валидации
checkpoint = torch.load(ckpt)
generator.load_state_dict(checkpoint["g_ema"])  # Подгрузка весов

# Определяем преобразования для изображения
transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Изменяем размер изображения до 256x256
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Нормализуем значения пикселей
])

# Загружаем в память предобученную модель энкодера pSp
model_path = '/content/encoder4editing/e4e_ffhq_encode.pt'  # Путь к весам энкодера
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']

opts['checkpoint_path'] = model_path
opts = Namespace(**opts)
net = pSp(opts)
net.eval()
net.to(device)
print('Model successfully loaded!')


# FACE ALIGNMENT
# Определяем тип эксперимента
experiment_type = "ffhq_encode"

# Задаем размер изображения для изменения
resize_dims = (256, 256)

def run_alignment(image_path):
  """Выравнивает лицо на изображении, используя dlib.

  Args:
      image_path: Путь к файлу изображения.

  Returns:
      Выровненное изображение в формате PIL.Image.
  """
  import dlib
  from utils.alignment import align_face  # Функция для выравнивания лица
  predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor)
  print("Aligned image has shape: {}".format(aligned_image.size))
  return aligned_image  # Возвращаем выровненное изображение

  # Определяем функцию для отображения результата рядом с исходным
def display_alongside_source_image(result_image, source_image):
    """Отображает результат рядом с исходным изображением.

    Args:
        result_image: Результирующее изображение (PIL.Image).
        source_image: Исходное изображение (PIL.Image).

    Returns:
        Объединенное изображение (PIL.Image).
    """
    res = np.concatenate([np.array(source_image.resize(resize_dims)),
                          np.array(result_image.resize(resize_dims))], axis=1)
    return Image.fromarray(res)


def run_on_batch(inputs, net):
    """Запускает модель на пакете данных.

    Args:
        inputs: Входные данные (тензор PyTorch).
        net: Модель pSp.

    Returns:
        Сгенерированные изображения и латентные векторы.
    """
    images, latents = net(inputs.to("cuda").float(), randomize_noise=False, return_latents=True)  # Запускаем модель
    return images, latents

# FACE EDITING
# Инициализируем редактор латентных векторов
from editings import latent_editor

editor = latent_editor.LatentEditor(net.decoder, False)

# Определяем доступные направления редактирования для InterFaceGAN
interfacegan_directions = {
    'ffhq_encode': {
        'age': 'editings/interfacegan_directions/age.pt',  # Направление для изменения возраста
        'smile': 'editings/interfacegan_directions/smile.pt',  # Направление для изменения улыбки
        'pose': 'editings/interfacegan_directions/pose.pt'  # Направление для изменения позы
    }
}

available_interfacegan_directions = None

available_interfacegan_directions = interfacegan_directions['ffhq_encode']  # Используем направления для "ffhq_encode"

print(list(available_interfacegan_directions.keys()))  # Выводим список ключей (направлений)

# ВОЗРАСТ [-10,10]
# As an example, we currently released the age and smile directions for the FFHQ StyleGAN Generator.

def change_all(image, age_factor, smile_factor, pose_factor):
  """Изменяет возраст, улыбку и поворот головы
  """
  if age_factor != 0:
    latents = get_latents(image)
    interfacegan_direction = torch.load(available_interfacegan_directions["age"]).cuda()
    image = editor.apply_interfacegan(latents, interfacegan_direction, factor=age_factor).resize(resize_dims)

  if smile_factor != 0:
    latents = get_latents(image)
    interfacegan_direction = torch.load(available_interfacegan_directions["smile"]).cuda()
    image = editor.apply_interfacegan(latents, interfacegan_direction, factor=smile_factor).resize(resize_dims)

  if pose_factor != 0:
    latents = get_latents(image)
    interfacegan_direction = torch.load(available_interfacegan_directions["pose"]).cuda()
    image = editor.apply_interfacegan(latents, interfacegan_direction, factor=pose_factor).resize(resize_dims)
  return image


def change_age(image, age_factor):
  """Изменяет возраст
  """
  latents = get_latents(image)
  interfacegan_direction = torch.load(available_interfacegan_directions["age"]).cuda()
  result = editor.apply_interfacegan(latents, interfacegan_direction, factor=age_factor).resize(resize_dims)
  return result

# SMILE [-10,10]
# As an example, we currently released the age and smile directions for the FFHQ StyleGAN Generator.
def change_smile(image, smile_factor):
  """Изменяет улыбку
  """
  latents = get_latents(image)
  interfacegan_direction = torch.load(available_interfacegan_directions["smile"]).cuda()
  result = editor.apply_interfacegan(latents, interfacegan_direction, factor=smile_factor).resize(resize_dims)
  return result

# POSE [-10,10]
def change_pose(image, pose_factor):
  """Изменяет поворот головы
  """
  latents = get_latents(image)
  interfacegan_direction = torch.load(available_interfacegan_directions["pose"]).cuda()
  result = editor.apply_interfacegan(latents, interfacegan_direction, factor=pose_factor).resize(resize_dims)
  return result

# Выравниваем лицо, если тип эксперимента - кодирование FFHQ
def align_resize_image(image):
  """Выравнивает и изменяет размер изображения для обработки нейросетью
  """
  if experiment_type == "ffhq_encode":
    input_image = run_alignment(image_path)
  else:
    input_image = original_image
  input_image.resize(resize_dims)
  return input_image

def get_latents(image):
  """Получает латентный вектор изображения для редактирования
  """
  image = transform(image)
  with torch.no_grad():
    images, latents = run_on_batch(image.unsqueeze(0), net)
    result_image, latent = images[0], latents[0]  # Извлекаем результат и латентный вектор
    return latents

Overwriting /content/face_editor.py


In [43]:
%%writefile /content/app.py
import streamlit as st
from PIL import Image
from io import BytesIO
import base64
import face_editor
import uuid

source_image = Image.open("/content/image.jpg")
edited_image = source_image

def edit_face():
  """Редактирует изображение
  """
  global source_image
  global edited_image
  global age_factor
  global smile_factor
  global pose_factor
  # with st.spinner(text="Doing magic..."):
  result_image = face_editor.change_all(source_image, age_factor=age_factor, smile_factor=smile_factor, pose_factor=pose_factor)
  edited_image = result_image
  col2.image(edited_image)
  # col2.download_button("Download image", convert_image(edited_image), "edited.png", "image/png", key=uuid.uuid4())

st.set_page_config(layout="wide", page_title="Face Editor")

st.write("## Load face image file and play with it!")
st.write(
    "(c) [@Maxim_Kan](https://t.me/Maxim_Kan) [(source)](https://github.com/kleorn/FaceEditor)"
)

age_factor = st.slider("Age modifier", -10, 10, 0)
smile_factor = st.slider("Smile modifier", -10, 10, 0)
pose_factor = st.slider("Head turn modifier", -10, 10, 0)

MAX_FILE_SIZE = 5 * 1024 * 1024  # 5MB

# Download the fixed image
def convert_image(img):
    """Конвертирует изображение для записи в файл
    """
    buf = BytesIO()
    img.save(buf, format="PNG")
    byte_im = buf.getvalue()
    return byte_im


def fix_image(upload):
    """Загружает изображение для редактирования
    """
    global source_image
    global edited_image
    source_image = Image.open(upload)
    col1.write("Original Image :camera:")
    col1.image(source_image)
    col1.button('Edit face!', on_click=edit_face)
    # edit_face()
    col2.write("Edited Image :wrench:")
    # col2.image(edited_image)
    st.sidebar.markdown("\n")
    return source_image

col1, col2 = st.columns(2)
my_upload = st.sidebar.file_uploader("Upload an image", type=["png", "jpg", "jpeg"])

if my_upload is not None:
    if my_upload.size > MAX_FILE_SIZE:
        st.error("The uploaded file is too large. Please upload an image smaller than 5MB.")
    else:
        source_image = fix_image(upload=my_upload)
else:
    fix_image("/content/image.jpg")

Overwriting /content/app.py


In [44]:
# Для успешной установки приходится запускать 2 раза, пока не появится строка "Run `npm audit` for details."
!npm install localtunnel
!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 822ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦⠙⠹⠸⠼
up to date, audited 23 packages in 732ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠴

In [45]:
!streamlit run /content/app.py &>/content/logs.txt &

In [46]:
# СКОПИРОВАТЬ IP-адрес, открыть ссылку (не IP!), а IP вставить как пароль
print('Скопируйте IP-адрес, откройте ссылку (не IP!), а IP вставьте, как пароль')
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

Скопируйте IP-адрес, откройте ссылку (не IP!), а IP вставьте, как пароль
34.124.229.249
⠙your url is: https://major-vans-doubt.loca.lt
